In [15]:
from requests import request
import requests
from requests.compat import urljoin, urlparse
from bs4 import BeautifulSoup
import re
from time import sleep
from openpyxl.workbook import Workbook # excel로 옮기기

In [16]:
import pandas as pd
import numpy as np

In [26]:
## while문 안에 for문도 넘 많아서시간이 오래 걸릴 거 같은데 어케 더 우아하게 수정할 수 없을까?

url_format = 'http://apis.data.go.kr/1741000/DisasterMsg2/getDisasterMsgList?ServiceKey=NhRuAoFGxRza0hrpsOJ1%2Fnvh6wrZSUse31YxriXHw273mOglft9Jy2u0JtJhzWqJ4vFUOHbaBWz0ObbzWR1lqw%3D%3D&type=xml&pageNo={page_num}&numOfRows=1000&flag=Y'
page_num = 1
a = 0

_date = []
_loc_id = []
_loc_name = []
_msg = []
_number = []

while True:
    url = url_format.format(page_num = page_num)
    resp = requests.get(url)
    html = resp.text
    
    soup = BeautifulSoup(html, 'html.parser')

    date = soup.find_all('create_date')
    loc_id = soup.find_all('location_id')
    loc_name = soup.find_all('location_name')
    msg = soup.find_all('msg')
    number = soup.find_all('md101_sn')
    
    a = len(_number)
    
    for content in number:
        _number.extend(content)
    for content in date:
        _date.extend(content)
    for content in loc_id:
        _loc_id.extend(content)
    for content in loc_name:
        _loc_name.extend(content)
    for content in msg:
        _msg.extend(content)

    page_num = page_num + 1
    if len(_number) == a:
        break
    
    sleep(200) ## 호출간격 180 정도로 설정 (너무 빠르면 호출에러 뜸)
    

KeyboardInterrupt: 

In [27]:
DisMsg = {}

DisMsg['Date'] = _date
DisMsg['LocationId'] = _loc_id
DisMsg['LocationName'] = _loc_name
DisMsg['Msg'] = _msg
DisMsg['Number'] = _number

df = pd.DataFrame(DisMsg) 

In [28]:
df

,Date,LocationId,LocationName,Msg,Number
0,2020/08/05 10:28:29,14,강원도 철원군,"[철원군청] 오늘 09시53분 집중호우로 한탄천 범람, 갈말읍 정연교-삼합교 일대 ...",42355
1,2020/08/05 10:24:14,235,충청남도 청양군,[청양군청] 오늘 강한 바람과 비가 예상됩니다.▲비닐하우스 고정▲배수구 정비▲위험수...,42354
2,2020/08/05 10:22:06,54,경상남도 거제시,"[거제시청] 22:30마산의료원이송(앰뷸런스)/자택 방역완료, 접촉자3명(남편, 자...",42353
3,2020/08/05 10:21:54,54,경상남도 거제시,[거제시청] 8.3(월) 9:00거제시보건소선별진료소(자차) 10:00자택 8.4(...,42352
4,2020/08/05 10:21:32,54,경상남도 거제시,[거제시청]19번 확진자 발생(프랑스국적 외국인/30대/여/프랑스체류 후 입국) 8...,42351
...,...,...,...,...,...
16995,2020/02/12 15:00:29,222,충청남도 전체,"[충남도청]신종코로나바이러스 예방을 위해 손씻기, 기침예절준수와 14일내 중국방문 ...",25360
16996,2020/02/12 14:17:26,196,전라남도 장성군,"[장성군]해외방문 후 발열, 호흡기 이상 발생시 장성군보건소(390-8333), 콜...",25359
16997,2020/02/12 13:54:04,216,전라북도 진안군,"[진안군청] 해외여행 14일 이내 발열(37.5도 이상), 호흡기 증상 시 진안군의...",25358
16998,2020/02/12 13:30:07,208,전라북도 부안군,[부안군청]부안군은 코로나바이러스 유입이없으니 안심하시기 바랍니다. 개인위생준수. ...,25357


In [30]:
## csv파일로 export

df.to_csv("msg_data", index = False)